# Burgers Equation
Equation:   $u_{t} + uu_{x}-\frac{0.01}{\pi}u_{xx} = 0$  
Boundary Conditions:  
$x \in [-1,1]$  $t \in [0,1]$  
$u(0,x)= -\sin(\pi x)$  
$u(t,-1)=u(t,1)=0$ 

In [1]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import grad
import scipy.io
from torch.utils.data import Dataset, DataLoader

In [2]:
!pip install pyDOE    #required for latin hypercube sampling of collocation points
from pyDOE import lhs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Data Prepocessing

In [3]:
nu = 0.01/np.pi
N_u = 100                                                                       #boundary points
N_f = 10000                                                                     #collacation points
layers = [2, 25, 25, 25, 25, 25, 25, 25, 25, 1]
data = scipy.io.loadmat('burgers_shock.mat')                                    #contains x,t and exact usol
t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T
X, T = np.meshgrid(x,t)
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))                  #2 columns containing x,t values
u_star = Exact.flatten()[:,None]                                                #1 column containing exact u values 
lb = X_star.min(0)                                                              #lower & upper bounds for x & t
ub = X_star.max(0) 
xx1 = np.hstack((X[0:1,:].T, T[0:1,:].T))
uu1 = Exact[0:1,:].T
xx2 = np.hstack((X[:,0:1], T[:,0:1]))
uu2 = Exact[:,0:1]
xx3 = np.hstack((X[:,-1:], T[:,-1:]))
uu3 = Exact[:,-1:]

X_u_train = np.vstack([xx1, xx2, xx3])
X_f_train = lb + (ub-lb)*lhs(2, N_f)                                            #Latin Hypercube Sampling method to generate collacation points
X_f_train = np.vstack((X_f_train, X_u_train))
u_train = np.vstack([uu1, uu2, uu3])
idx = np.random.choice(X_u_train.shape[0], N_u, replace=False)                  #Randomly choosing 100 training points
X_u_train = X_u_train[idx, :]
u_train = u_train[idx,:]

X_u = torch.from_numpy(X_u_train[:,0:1]).float()         #x boundary points 
T_u = torch.from_numpy(X_u_train[:,1:2]).float()         #t boundary points 
X_f = torch.tensor(X_f_train[:,0:1], requires_grad = True).float()        #x collocation points
T_f = torch.tensor(X_f_train[:,1:2], requires_grad = True).float()         #t collocation points
u_train = torch.from_numpy(u_train).float()

## PINN (Formulation \& Implementation)

In [4]:
class PINN(nn.Module):

  def __init__(self, layers):
    super(PINN, self).__init__()
    self.layers = nn.ModuleList()
    for i, j in zip(layers, layers[1:]):
      linear = nn.Linear(i, j)
      nn.init.xavier_normal_(linear.weight.data, gain = 1.0)
      nn.init.zeros_(linear.bias.data)
      self.layers.append(linear)
  
  def forward(self, x):
    L = len(self.layers)
    for l, transform in enumerate(self.layers):
      if l < L-1:
        x = torch.tanh(transform(x))
      else:
        x = transform(x)
    return x  

In [5]:
class PINN_Run():

  def __init__(self, X_f, T_f, nu, X_u, T_u, model, u_train):
    self.X_f = X_f
    self.T_f = T_f
    self.nu = nu
    self.X_u = X_u
    self.T_u = T_u
    self.u_train = u_train
    self.model = model
    self.optimizer = torch.optim.LBFGS(
            self.model.parameters(), 
            lr = 1.0, 
            max_iter=20000, 
            max_eval=20000, 
            history_size=40,
            tolerance_grad=1e-6, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"       
        )
    self.i = 0

  def residual_loss(self):
    xf = torch.cat([self.X_f, self.T_f], axis=1)
    uf = self.model(xf)
    u_x = grad(uf.sum(), self.X_f, retain_graph = True, create_graph = True)[0]
    u_xx = grad(u_x.sum(), self.X_f, retain_graph = True, create_graph = True)[0]
    u_t = grad(uf.sum(), self.T_f, retain_graph = True, create_graph = True)[0]
    f = u_t + uf*u_x - self.nu*u_xx 
    return torch.mean(torch.square(f))
  
  def closure(self):
    self.model.train()
    mse = nn.MSELoss()
    self.optimizer.zero_grad()
    yhat = self.model(torch.cat([self.X_u, self.T_u], axis=1).float())
    loss1 = mse(yhat, self.u_train)
    loss2 = self.residual_loss()
    loss = loss1 + loss2
    loss.backward()
    if self.i % 100 == 0:
      print('Epoch:', self.i, 'Loss: %.5e' % (loss.item()))
    self.i += 1
    return loss
  
  def train_(self):
    self.optimizer.step(self.closure)  

In [6]:
pinn_model = PINN(layers)
pinn_ = PINN_Run(X_f, T_f, nu, X_u, T_u, pinn_model, u_train)

In [7]:
pinn_.train_()

Epoch: 0 Loss: 3.68430e-01
Epoch: 100 Loss: 7.57236e-02
Epoch: 200 Loss: 4.54286e-02
Epoch: 300 Loss: 1.58204e-02
Epoch: 400 Loss: 4.87081e-03
Epoch: 500 Loss: 1.47870e-03
Epoch: 600 Loss: 7.14642e-04
Epoch: 700 Loss: 4.53142e-04
Epoch: 800 Loss: 3.39726e-04
Epoch: 900 Loss: 2.79447e-04
Epoch: 1000 Loss: 2.28179e-04
Epoch: 1100 Loss: 1.90270e-04
Epoch: 1200 Loss: 1.63677e-04
Epoch: 1300 Loss: 1.42596e-04
Epoch: 1400 Loss: 1.23679e-04
Epoch: 1500 Loss: 1.11510e-04
Epoch: 1600 Loss: 9.92593e-05
Epoch: 1700 Loss: 8.70449e-05
Epoch: 1800 Loss: 7.51133e-05
Epoch: 1900 Loss: 6.88663e-05
Epoch: 2000 Loss: 6.29026e-05
Epoch: 2100 Loss: 5.63711e-05
Epoch: 2200 Loss: 5.26291e-05
Epoch: 2300 Loss: 4.92547e-05
Epoch: 2400 Loss: 4.54786e-05
Epoch: 2500 Loss: 4.12262e-05
Epoch: 2600 Loss: 3.75024e-05
Epoch: 2700 Loss: 3.46659e-05
Epoch: 2800 Loss: 3.28091e-05
Epoch: 2900 Loss: 3.12918e-05
Epoch: 3000 Loss: 3.00010e-05
Epoch: 3100 Loss: 2.83870e-05
Epoch: 3200 Loss: 2.67661e-05
Epoch: 3300 Loss: 2.53

In [8]:
pinn_model.eval()
u_pinn = pinn_model(torch.from_numpy(X_star).float())
table = np.hstack((u_pinn.detach().numpy(), u_star))
print('Predicted   -   Actual')
print(table[10010:10020])

Predicted   -   Actual
[[0.28538823 0.28533147]
 [0.2961781  0.29615185]
 [0.30694997 0.30695429]
 [0.31770301 0.31773801]
 [0.32843637 0.32850224]
 [0.33914948 0.33924619]
 [0.34984195 0.34996906]
 [0.36051273 0.36067002]
 [0.37116086 0.37134826]
 [0.38178539 0.38200292]]
